In [12]:
import sys
sys.path.insert(1, '/Users/linusrandud/Documents/UoM/ERP/MscDissertation/Deep-Opt')

In [13]:
import torch

from COProblems.MKP import MKP
from COProblems.QUBO import QUBO
from Models.DOAE import DOAE
from OptimAE import OptimAEHandler

In [14]:
# Highly recommended to keep as cpu for problems of size <= 100
device="cpu"
print(device)
device = torch.device(device)

cpu


In [15]:
problem = MKP("/Users/linusrandud/Documents/UoM/ERP/MscDissertation/data/mkp/problems30d.txt", "/Users/linusrandud/Documents/UoM/ERP/MscDissertation/data/mkp/fitnesses30d.txt", 12, device)

This instance has 100 items and 30 dimensions
Max possible fitness for this instance: 41560.0


In [5]:
problem

In [6]:
dir(problem)

['A',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'b',
 'c',
 'device',
 'fitness',
 'get_utility_order',
 'is_valid',
 'max_fitness',
 'random_solution',
 'repair',
 'utility']

In [7]:
problem.b

tensor([26260., 27025., 25353., 20423., 21801., 27803., 25571., 25057., 23284.,
        24242., 23950., 25767., 27937., 24260., 28375., 24405., 24963., 24938.,
        25390., 26130., 22794., 27212., 22649., 25417., 22954., 25662., 24486.,
        24032., 24052., 25164.])

In [8]:
problem.c

tensor([ 578.,  735.,  972.,  568.,  671.,  948.,  742.,  579.,  546.,  596.,
         891.,  779.,  755.,  630.,  981.,  641.,  654.,  627., 1041.,  819.,
         811.,  488.,  790.,  613.,  879.,  482., 1031.,  609.,  870.,  669.,
        1008.,  892.,  676.,  914., 1007.,  920.,  739.,  960.,  518.,  524.,
         658.,  751.,  804.,  619.,  519.,  898.,  925.,  957.,  899.,  981.,
         756.,  697.,  796.,  807.,  916.,  477.,  855.,  559.,  823.,  690.,
         936.,  759.,  781.,  594.,  723.,  746.,  549.,  870.,  716.,  603.,
         619.,  649.,  763.,  699.,  748.,  487.,  948.,  620.,  889.,  680.,
         666.,  617.,  914.,  942.,  781.,  902.,  814.,  628.,  561.,  655.,
         906.,  585.,  850.,  933.,  727.,  403.,  593.,  762.,  619.,  588.])

In [9]:
problem.A

tensor([[ 67., 720., 669.,  ..., 239., 640., 658.],
        [451., 818., 367.,  ...,  23., 693., 744.],
        [845., 975., 987.,  ..., 178., 227., 692.],
        ...,
        [907., 816., 503.,  ..., 945., 594., 310.],
        [348., 649., 992.,  ..., 892., 756., 190.],
        [711., 947., 530.,  ..., 774., 779., 596.]])

In [10]:
problem.device

device(type='cpu')

In [11]:
# Opening .txt file to read raw data of an instance
name = '/Users/linusrandud/Documents/UoM/ERP/MscDissertation/data/mkp/problems30d.txt'

file = open(str(name), 'r')
x = []
for line in file:
    split_line = line.split()
    for i in range(len(split_line)):
        x.append(split_line[i])
file.close()

In [12]:
x, len(x)

(['30',
  '100',
  '30',
  '0',
  '1002',
  '889',
  '774',
  '746',
  '873',
  '916',
  '731',
  '660',
  '601',
  '792',
  '647',
  '623',
  '566',
  '574',
  '953',
  '922',
  '927',
  '670',
  '814',
  '661',
  '590',
  '1013',
  '640',
  '911',
  '650',
  '670',
  '1067',
  '988',
  '765',
  '955',
  '857',
  '943',
  '502',
  '584',
  '761',
  '599',
  '507',
  '804',
  '462',
  '558',
  '648',
  '484',
  '898',
  '1052',
  '568',
  '617',
  '690',
  '665',
  '754',
  '604',
  '564',
  '498',
  '804',
  '496',
  '1013',
  '834',
  '561',
  '873',
  '859',
  '578',
  '534',
  '819',
  '619',
  '953',
  '497',
  '843',
  '800',
  '934',
  '757',
  '570',
  '946',
  '691',
  '1006',
  '1027',
  '714',
  '630',
  '565',
  '955',
  '828',
  '564',
  '601',
  '631',
  '823',
  '657',
  '617',
  '674',
  '624',
  '647',
  '506',
  '1020',
  '870',
  '929',
  '921',
  '484',
  '867',
  '926',
  '748',
  '583',
  '882',
  '586',
  '193',
  '665',
  '929',
  '958',
  '962',
  '186',
  '666

In [13]:
id = 12
i = 1
cur_id = 0
while cur_id < id:
    num_columns = int(x[i])
    num_rows = int(x[i+1])
    i += num_columns + (num_columns*num_rows) + num_rows + 3
    cur_id += 1
num_columns = int(x.pop(i))
num_rows = int(x.pop(i))
best = int(x.pop(i))

In [14]:
num_columns, num_rows, best

(100, 30, 0)

In [15]:
len(x)

93988

In [16]:
c = torch.Tensor([float(x.pop(i)) for _ in range(num_columns)])

In [17]:
c

tensor([ 578.,  735.,  972.,  568.,  671.,  948.,  742.,  579.,  546.,  596.,
         891.,  779.,  755.,  630.,  981.,  641.,  654.,  627., 1041.,  819.,
         811.,  488.,  790.,  613.,  879.,  482., 1031.,  609.,  870.,  669.,
        1008.,  892.,  676.,  914., 1007.,  920.,  739.,  960.,  518.,  524.,
         658.,  751.,  804.,  619.,  519.,  898.,  925.,  957.,  899.,  981.,
         756.,  697.,  796.,  807.,  916.,  477.,  855.,  559.,  823.,  690.,
         936.,  759.,  781.,  594.,  723.,  746.,  549.,  870.,  716.,  603.,
         619.,  649.,  763.,  699.,  748.,  487.,  948.,  620.,  889.,  680.,
         666.,  617.,  914.,  942.,  781.,  902.,  814.,  628.,  561.,  655.,
         906.,  585.,  850.,  933.,  727.,  403.,  593.,  762.,  619.,  588.])

In [18]:
len(x)

93888

In [19]:
# Populating A matrix (size NumRows * NumColumns)
const_coef = torch.Tensor([float(x.pop(i)) for _ in range(int(num_rows * num_columns))])           
A = torch.reshape(const_coef, (num_rows, num_columns)) # reshaping the 1-d ConstCoef into A

In [20]:
const_coef

tensor([ 67., 720., 669.,  ..., 774., 779., 596.])

In [21]:
A.shape

torch.Size([30, 100])

In [22]:
A[0]

tensor([ 67., 720., 669., 404., 341., 823., 775., 734., 368., 160., 864., 117.,
        104., 818., 741., 109., 895., 967., 824., 212., 968., 970., 840., 787.,
        884., 486., 953., 110., 659., 638., 435.,  38., 898., 380., 611., 330.,
        983., 490., 724., 282., 262., 322., 443., 271., 265., 382., 677., 588.,
        864., 831., 778., 493., 742., 766., 350., 217., 943., 699., 995., 681.,
        275., 681., 320.,  86., 852., 102., 503., 132., 479., 116., 649., 193.,
        564., 441., 610., 219., 613., 662., 597., 956., 408., 489., 613., 134.,
        221., 915., 385.,  78., 614., 441.,  13., 555., 593., 315., 719., 532.,
        160., 239., 640., 658.])

In [23]:
num_rows

30

In [24]:
# Populating the RHS
b = torch.Tensor([float(x.pop(i)) for _ in range(int(num_rows))])

In [25]:
b

tensor([26260., 27025., 25353., 20423., 21801., 27803., 25571., 25057., 23284.,
        24242., 23950., 25767., 27937., 24260., 28375., 24405., 24963., 24938.,
        25390., 26130., 22794., 27212., 22649., 25417., 22954., 25662., 24486.,
        24032., 24052., 25164.])

In [26]:
name = '/Users/linusrandud/Documents/UoM/ERP/MscDissertation/data/mkp/fitnesses30d.txt'

file = open(str(name), 'r')
max_fitness = 0
for i, line in enumerate(file):
    if i == id:
        max_fitness = float(line)
        break
file.close()

In [27]:
max_fitness

41560.0

In [28]:
x.matmul(A.T)

AttributeError: 'list' object has no attribute 'matmul'

In [16]:
import torch

from COProblems.MKP import MKP
from COProblems.QUBO import QUBO
from Models.DOAE import DOAE
from OptimAE import OptimAEHandler

# Highly recommended to keep as cpu for problems of size <= 100
device = "cuda" if torch.cuda.is_available() else "cpu"
device="cpu"
print(device)
device = torch.device(device)

change_tolerance = 100
problem_size = 100
pop_size = 200
# problem = MKP("COProblems\\mkp\\problems30d.txt", "COProblems\\mkp\\fitnesses30d.txt", 12, device)
#problem = QUBO("COProblems\\qubo\\bqp500.txt", 0, device)

dropout_prob = 0.2
# 1000 bit QUBO l1 and l2:
# l1_coef = 0.0000025
# l2_coef = 0.0000025
# 500 bit QUBO l1 and l2:
# l1_coef = 0.000005
# l2_coef = 0.000005
# 100 bit l1 and l2
l1_coef = 0.0001
l2_coef = 0.0001
lr = 0.002
compression_ratio = 0.8
model = DOAE(problem_size, dropout_prob, device)
hidden_size = problem_size
handler = OptimAEHandler(model, problem, device)

population, fitnesses = handler.generate_population(pop_size)
population, fitnesses, _, _ = handler.hillclimb(population, fitnesses, change_tolerance)
handler.print_statistics(fitnesses)

total_eval = 0
max_depth = 6
depth = 0

while True:
    if depth < max_depth:
        hidden_size = round(hidden_size * compression_ratio)
        model.transition(hidden_size)
        depth += 1
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_coef)
    print("Learning from population")
    # Learing with the entire population as a batch is technically not the best from a machine learning perspective,
    # but does not seem to have a massive impact on solution quality whilst also increasing learning speed significantly.
    handler.learn_from_population(population, optimizer, l1_coef=l1_coef, batch_size=pop_size)
    print("Optimising population")
    population, fitnesses, evaluations, done = handler.optimise_solutions(
        population, fitnesses, change_tolerance, encode=True, repair_solutions=True, deepest_only=False
    )
    handler.print_statistics(fitnesses)
    total_eval += evaluations
    print("Evaluations: {}".format(total_eval))
    
    # Uncomment lines below to print out best solution at every transition
    # best_i = torch.argmax(fitnesses)
    # print("Best solution - fitness = {}".format(fitnesses[best_i].item()))
    # print(population[best_i].tolist())
    if done:
        break

cpu
Max pop fitness: 36696.0, Mean pop fitness : 33667.515625
Learning from population
Optimising population
Max pop fitness: 41073.0, Mean pop fitness : 40493.703125
Evaluations: 49434
Learning from population
Optimising population
Max pop fitness: 41282.0, Mean pop fitness : 40849.21875
Evaluations: 105421
Learning from population
Optimising population
Max pop fitness: 41371.0, Mean pop fitness : 40993.328125
Evaluations: 176835
Learning from population
Optimising population
Max pop fitness: 41371.0, Mean pop fitness : 41086.3359375
Evaluations: 265773
Learning from population
Optimising population
Max pop fitness: 41417.0, Mean pop fitness : 41159.28125
Evaluations: 374119
Learning from population
Optimising population
Max pop fitness: 41417.0, Mean pop fitness : 41210.9296875
Evaluations: 501331
Learning from population
Optimising population
Max pop fitness: 41417.0, Mean pop fitness : 41240.0
Evaluations: 627167
Learning from population
Optimising population
Max pop fitness: 41417

KeyboardInterrupt: 

In [17]:
population

tensor([[-1., -1.,  1.,  ..., -1., -1., -1.],
        [-1., -1.,  1.,  ..., -1., -1., -1.],
        [-1., -1.,  1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1.,  1.,  ..., -1., -1., -1.],
        [-1., -1.,  1.,  ..., -1., -1., -1.],
        [-1., -1.,  1.,  ..., -1., -1., -1.]])

In [18]:
population.shape

torch.Size([200, 100])

In [19]:
population[0]

tensor([-1., -1.,  1., -1., -1.,  1., -1., -1., -1.,  1.,  1.,  1.,  1., -1.,
         1.,  1., -1., -1.,  1.,  1., -1., -1., -1., -1., -1., -1.,  1., -1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1.,  1.,
         1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1., -1.,
         1., -1.,  1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,  1.,  1.,
        -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1., -1., -1., -1.,  1.,  1.,
        -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1., -1., -1.,
        -1., -1.])

In [20]:
fitnesses

tensor([41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518.,
        41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518.,
        41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518.,
        41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518.,
        41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518.,
        41518., 41518., 41518., 41518., 41518., 41518., 41399., 41518., 41518.,
        41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518.,
        41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518.,
        41518., 41518., 41518., 41518., 41518., 41518., 41371., 41518., 41518.,
        41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518.,
        41518., 41518., 41518., 41440., 41518., 41518., 41518., 41518., 41518.,
        41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518., 41518.,
        41518., 41518., 41518., 41518., 

In [9]:
evaluations

60600